# Logistic Regression for Sentiment Analysis

Adapted from http://nbviewer.jupyter.org/github/rasbt/pattern_classification/blob/master/machine_learning/scikit-learn/outofcore_modelpersistence.ipynb

<br>
<br>

## The IMDb Movie Review Dataset

In this section, we will train a simple logistic regression model to classify movie reviews from the 50k IMDb review dataset that has been collected by Maas et. al.

> AL Maas, RE Daly, PT Pham, D Huang, AY Ng, and C Potts. Learning word vectors for sentiment analysis. In Proceedings of the 49th Annual Meeting of the Association for Computational Lin- guistics: Human Language Technologies, pages 142–150, Portland, Oregon, USA, June 2011. Association for Computational Linguistics

[Source: http://ai.stanford.edu/~amaas/data/sentiment/]

The dataset consists of 50,000 movie reviews from the original "train" and "test" subdirectories. The class labels are binary (1=positive and 0=negative) and contain 25,000 positive and 25,000 negative movie reviews, respectively.
For simplicity, I assembled the reviews in a single CSV file.


In [1]:
import pandas as pd
# if you want to download the original file:
#df = pd.read_csv('https://raw.githubusercontent.com/rasbt/pattern_classification/master/data/50k_imdb_movie_reviews.csv')
# otherwise load local file
df = pd.read_csv('shuffled_movie_data.csv')
df.tail()

,review,sentiment
49995,This movie is probably for you. It had an over...,1
49996,From the start you will like Sam Elliott's cha...,1
49997,"Good Movie, acting was terrific especially fro...",1
49998,"Not the best of the WIP's, but not the worst e...",0
49999,Did the first travesty actually make money? Th...,0


Let us shuffle the class labels.

In [5]:
import numpy as np
## uncomment these lines if you have dowloaded the original file:
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df[['review', 'sentiment']].to_csv('shuffled_movie_data.csv', index=False)
df.tail()

,review,sentiment
14703,This proved to be a rare case of a poliziottes...,0
39659,This is a superbly imaginative low budget Sci-...,1
16392,I couldn't believe my eyes when I watched Nure...,0
43065,Some people have made a point of dissing this ...,0
33553,I have to finish watching a movie once I start...,0


<br>
<br>

## Preprocessing Text Data

Now, let us define a simple `tokenizer` that splits the text into individual word tokens. Furthermore, we will use some simple regular expression to remove HTML markup and all non-letter characters but "emoticons," convert the text to lower case, remove stopwords, and apply the Porter stemming algorithm to convert the words into their root form.

In [6]:
import numpy as np
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text

Let's give it at try:

In [7]:
tokenizer('This :) is a <a> test! :-)</br>')

['test', ':)', ':)']

## Learning (SciKit)

First, we define a generator that returns the document body and the corresponding class label:

In [8]:
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

To conform that the `stream_docs` function fetches the documents as intended, let us execute the following code snippet before we implement the `get_minibatch` function:

In [9]:
next(stream_docs(path='shuffled_movie_data.csv'))

('"Well, you\'d better if you plan on sitting through this amateurish, bland, and pokey flick about a middle-aged widowed mom who has a little more in common with her young adult or old teen daughter than she would like. Set in Tunis, mom piddles around the flat, gets antsy, and decides, albeit reluctantly (she just can\'t help herself), to don the costume and dance in a local cabaret. Meanwhile her daughter is taking dancing lessons. The common denominator is a Tunisian band drummer. This film is so full of filler I watched the DVD at x2 and read the subtitles, fast forwarding through much of the very ordinary dancing and loooong shots of walking (they walk everywhere) and more walking and just plain dawdling at x4 just to get though this boring, uneventful, low budget flick which some how garnered some pretty good critical plaudits. Go figure. (C-)"',
 0)

After we confirmed that our `stream_docs` functions works, we will now implement a `get_minibatch` function to fetch a specified number (`size`) of documents:

In [10]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    for _ in range(size):
        text, label = next(doc_stream)
        docs.append(text)
        y.append(label)
    return docs, y

Next, we will make use of the "hashing trick" through scikit-learns [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) to create a bag-of-words model of our documents. Details of the bag-of-words model for document classification can be found at  [Naive Bayes and Text Classification I - Introduction and Theory](http://arxiv.org/abs/1410.5329).

In [12]:
from sklearn.feature_extraction.text import HashingVectorizer
vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

# Excercise 1: define new features according to https://web.stanford.edu/~jurafsky/slp3/5.pdf



Using the [SGDClassifier]() from scikit-learn, we will can instanciate a logistic regression classifier that learns from the documents incrementally using stochastic gradient descent. 

In [13]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path='shuffled_movie_data.csv')

# Excercise 2: implement a Logistic Regression classifier, using regularization, according to https://web.stanford.edu/~jurafsky/slp3/5.pdf

def sigmoid(x):
    return 1.0 / (1 + np.exp(-x))

def stocashticGradientDescent(X,Y,theta,learning_rate=0.01,iterations=10):
    m = len(Y)
    cost_history = np.zeros(iterations)
    
    for it in range(iterations):
        for i in range(m):
            rand_ind = np.random.randint(0,m)
            X_i = X[rand_ind,:].reshape(1,X.shape[1])
            Y_i = Y[rand_ind].reshape(1,1)
            prediction = sigmoid(np.dot(X_i,theta)+b)
            g = ( X_i.T.dot((prediction - Y_i)))
            print(g)
            theta = theta - learning_rate*g
        
    return theta

In [25]:
#import pyprind
#pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])

#for _ in range(45):
X_train, Y_train = get_minibatch(doc_stream, size=1000)
print(X_train[0])
print(Y_train[0])
X_train = vect.transform(X_train)
type(X_train)
    #clf.partial_fit(X_train, Y_train, classes=classes)
    ##pbar.update()

"Tainted look at kibbutz life<br /><br />This film is less a cultural story about a boy's life in a kibbutz, but the deliberate demonization of kibbutz life in general. In the first two minutes of the movie, the milk man in charge of the cows rapes one of his calves. And it's all downhill from there in terms of the characters representing typical ""kibbutznikim."" Besides the two main characters, a clinically depressed woman and her young son, every one else in the kibbutz is a gross caricature of wellevil. <br /><br />The story centers on how the kibbutz, like some sort of cult, slowly drags the mother and son deeper into despair and what inevitably follows. There is no happiness, no joy, no laughter in this kibbutz. Every character/situation represents a different horrific human vice like misogyny, hypocrisy, violence, cultism, repression etc. For example, while the protagonist is a strikingly handsome European looking 12 year old boy  his older brother is a typical kibbutz youth c

scipy.sparse.csr.csr_matrix

Depending on your machine, it will take about 2-3 minutes to stream the documents and learn the weights for the logistic regression model to classify "new" movie reviews. Executing the preceding code, we used the first 45,000 movie reviews to train the classifier, which means that we have 5,000 reviews left for testing:

In [ ]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
print(type(X_test))
X_test = vect.transform(X_test)
print(type(X_test))
print('Accuracy: %.3f' % clf.score(X_test, y_test))

I think that the predictive performance, an accuracy of ~87%, is quite "reasonable" given that we "only" used the default parameters and didn't do any hyperparameter optimization. 

After we estimated the model perfomance, let us use those last 5,000 test samples to update our model.

In [ ]:
clf = clf.partial_fit(X_test, y_test)

<br>
<br>

# Model Persistence

In the previous section, we successfully trained a model to predict the sentiment of a movie review. Unfortunately, if we'd close this IPython notebook at this point, we'd have to go through the whole learning process again and again if we'd want to make a prediction on "new data."

So, to reuse this model, we could use the [`pickle`](https://docs.python.org/3.5/library/pickle.html) module to "serialize a Python object structure". Or even better, we could use the [`joblib`](https://pypi.python.org/pypi/joblib) library, which handles large NumPy arrays more efficiently.

To install:
conda install -c anaconda joblib

In [ ]:
import joblib
import os
if not os.path.exists('./pkl_objects'):
    os.mkdir('./pkl_objects')
    
joblib.dump(vect, './vectorizer.pkl')
joblib.dump(clf, './clf.pkl')

Using the code above, we "pickled" the `HashingVectorizer` and the `SGDClassifier` so that we can re-use those objects later. However, `pickle` and `joblib` have a known issue with `pickling` objects or functions from a `__main__` block and we'd get an `AttributeError: Can't get attribute [x] on <module '__main__'>` if we'd unpickle it later. Thus, to pickle the `tokenizer` function, we can write it to a file and import it to get the `namespace` "right".

In [ ]:
%%writefile tokenizer.py
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text

In [ ]:
from tokenizer import tokenizer
joblib.dump(tokenizer, './tokenizer.pkl')

Now, let us restart this IPython notebook and check if the we can load our serialized objects:

In [ ]:
import joblib
tokenizer = joblib.load('./tokenizer.pkl')
vect = joblib.load('./vectorizer.pkl')
clf = joblib.load('./clf.pkl')

After loading the `tokenizer`, `HashingVectorizer`, and the tranined logistic regression model, we can use it to make predictions on new data, which can be useful, for example, if we'd want to embed our classifier into a web application -- a topic for another IPython notebook.

In [ ]:
example = ['I did not like this movie']
X = vect.transform(example)
clf.predict(X)

In [ ]:
example = ['I loved this movie']
X = vect.transform(example)
clf.predict(X)